In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install torch torchvision torchaudio --upgrade
!pip install pillow numpy matplotlib
!pip install ultralytics

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import numpy as np
import os
import copy
import time
import shutil
from PIL import Image
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision.datasets.folder import IMG_EXTENSIONS
from ultralytics import YOLO

In [ ]:
dataset_path = "/content/drive/MyDrive/Colab Notebooks/Plant Disease Dataset"
train_dir = dataset_path + "/train"
validation_dir = dataset_path + "/validation"


In [ ]:
# This copies the dataset from Drive to the Colab instance's local temporary storage (/content/)
# Path on Google Drive
source_dir = "/content/drive/MyDrive/Colab Notebooks/Plant Disease Dataset"
# Path on local Colab machine
local_dir = "/content/plant_data"

if not os.path.exists(local_dir):
    shutil.copytree(source_dir, local_dir)
print("Copy complete!")

dataset_path = local_dir
train_dir = dataset_path + "/train"
validation_dir = dataset_path + "/validation"

Copy complete!


In [ ]:
# Define standard ResNet transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),  # ResNet expects 224x224
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_dataset = ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = ImageFolder(validation_dir, transform=data_transforms['validation'])

batch_size = 64

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

In [ ]:
print(torch.cuda.get_device_name(0))

NVIDIA A100-SXM4-40GB


In [ ]:
# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze parameters
for param in model.parameters():
    param.requires_grad = False

# Calculate num_classes from the DATASET, not the directory string
num_classes = len(train_dataset.classes)
print(f"Detected {num_classes} classes.")

# Replace final layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model created and moved to {device}")

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 118MB/s]


Detected 23 classes.
Model created and moved to cuda


In [ ]:
# 1. Define the Loss Function
# CrossEntropyLoss is the standard for multi-class classification
criterion = nn.CrossEntropyLoss()

# 2. Define the Optimizer
# We only pass model.fc.parameters() because we froze the rest of the network
# Learning rate (lr) of 0.001 is a good starting point
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

print("Loss function and Optimizer defined.")

Loss function and Optimizer defined.


In [ ]:
# ResNet50
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    start_time = time.time()
    # Store history to plot later
    train_acc_history = []
    val_acc_history = []

    print(f"Starting training on {device}...")

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                data_loader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                data_loader = val_loader

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data (batches)
            for inputs, labels in data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward Pass
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # Backward Pass + Optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = running_corrects.double() / len(data_loader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Save history
            if phase == 'train':
                train_acc_history.append(epoch_acc.item())
            else:
                val_acc_history.append(epoch_acc.item())

        print()

    time_elapsed = time.time() - start_time
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    return model, train_acc_history, val_acc_history

# 3. Run the training
trained_model, train_acc, val_acc = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)

Starting training on cuda...
Epoch 1/20
----------
train Loss: 0.1268 Acc: 0.9595
val Loss: 0.1120 Acc: 0.9639

Epoch 2/20
----------
train Loss: 0.1172 Acc: 0.9632
val Loss: 0.1028 Acc: 0.9697

Epoch 3/20
----------
train Loss: 0.1053 Acc: 0.9665
val Loss: 0.0936 Acc: 0.9711

Epoch 4/20
----------
train Loss: 0.0988 Acc: 0.9679
val Loss: 0.0838 Acc: 0.9745

Epoch 5/20
----------
train Loss: 0.0955 Acc: 0.9689
val Loss: 0.0853 Acc: 0.9742

Epoch 6/20
----------
train Loss: 0.0916 Acc: 0.9691
val Loss: 0.0866 Acc: 0.9721

Epoch 7/20
----------
train Loss: 0.0882 Acc: 0.9698
val Loss: 0.0781 Acc: 0.9747

Epoch 8/20
----------
train Loss: 0.0846 Acc: 0.9715
val Loss: 0.0734 Acc: 0.9780

Epoch 9/20
----------
train Loss: 0.0777 Acc: 0.9742


In [ ]:
# Define the path in your Google Drive
save_path = '/content/drive/MyDrive/Colab Notebooks/Plant Disease Dataset/models/plant_disease_resnet50.pth'

# Save the model weights
torch.save(model.state_dict(), save_path)

print(f"Model saved successfully to: {save_path}")

Model saved successfully to: /content/drive/MyDrive/Colab Notebooks/Plant Disease Dataset/models/plant_disease_resnet50.pth


In [ ]:
# YOLOv8
dataset_path = "/content/plant_data"

model = YOLO("yolov8s-cls.pt")

# Train YOLO classification model
results = model.train(
    data=dataset_path,
    epochs=20,
    imgsz=224,
    batch=64,
    lr0=0.001,
    device=0  # GPU
)

print("Training finished!")

save_path = "/content/drive/MyDrive/Colab Notebooks/Plant Disease Dataset/models"
os.makedirs(save_path, exist_ok=True)

# Define where YOLO saved the weights
# Note: The files are ALWAYS named 'best.pt' and 'last.pt' inside that folder
local_best = "runs/classify/train/weights/best.pt"
local_last = "runs/classify/train/weights/last.pt"

# Copy and Rename to Drive
# We rename them to 'yolo_best.pt' while copying so they don't get mixed up in Drive
shutil.copy(local_best, os.path.join(save_path, "yolo_best.pt"))
shutil.copy(local_last, os.path.join(save_path, "yolo_last.pt"))

print(f"Models saved to: {save_path}")

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/plant_data, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

In [ ]:
# MobileNetV3-Small
model_mnet = models.mobilenet_v3_small(pretrained=True)
for param in model_mnet.features.parameters():
    param.requires_grad = False

# Compute number of classes dynamically
num_classes = len(train_dataset.classes)
print(f"Detected {num_classes} classes for MobileNetV3-Small.")

# Replace classifier head
# Default structure: model_mnet.classifier = Sequential(...)
in_features = model_mnet.classifier[3].in_features
model_mnet.classifier[3] = nn.Linear(in_features, num_classes)

# Move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_mnet = model_mnet.to(device)
print("MobileNetV3-Small model created and moved to", device)

# --- Loss & Optimizer ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_mnet.classifier.parameters(), lr=0.001)

print("Loss function and optimizer ready.")

In [ ]:
def train_mobilenet(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    start_time = time.time()
    best_acc = 0.0
    save_dir = "runs/classify/mobilenet/weights"

    os.makedirs(save_dir, exist_ok=True)

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
                loader = train_loader
            else:
                model.eval()
                loader = val_loader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(loader.dataset)
            epoch_acc = running_corrects.double() / len(loader.dataset)

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # Save best model (only from validation phase)
            if phase == "val":
                # Save "last.pt" every epoch
                torch.save(model.state_dict(), os.path.join(save_dir, "last.pt"))

                # Save "best.pt" if accuracy improved
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), os.path.join(save_dir, "best.pt"))
                    print("Best model updated!")

    total_time = time.time() - start_time
    print(f"\nTraining completed in {total_time//60:.0f}m {total_time%60:.0f}s")
    print(f"Best validation accuracy: {best_acc:.4f}")
    print(f"Models saved to {save_dir}")

    return model


# Run Training
mobilenet_model = train_mobilenet(
    model_mnet, train_loader, val_loader, criterion, optimizer, num_epochs=20
)

# Save Model
drive_save_dir = "/content/drive/MyDrive/Colab Notebooks/Plant Disease Dataset/models/mobilenetv3"
os.makedirs(drive_save_dir, exist_ok=True)

shutil.copy("runs/classify/mobilenet/weights/best.pt", os.path.join(drive_save_dir, "mobilenet_best.pt"))
shutil.copy("runs/classify/mobilenet/weights/last.pt", os.path.join(drive_save_dir, "mobilenet_last.pt"))

print(f"\nMobileNet models copied to Drive folder:\n{drive_save_dir}")

In [ ]:
# 1. Setup Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Define Transforms for EfficientNetV2
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((384, 384)), # Resize to 384
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize((384, 384)), # Resize to 384
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

dataset_path = "/content/plant_data"
train_dataset = ImageFolder(dataset_path + "/train", transform=data_transforms['train'])
val_dataset = ImageFolder(dataset_path + "/validation", transform=data_transforms['validation'])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# 4. Setup Model
weights = models.EfficientNet_V2_S_Weights.IMAGENET1K_V1
model = models.efficientnet_v2_s(weights=weights)

# FREEZE the backbone (Feature Extractor)
for param in model.parameters():
    param.requires_grad = False

# Replace the Classifier Head
# EfficientNet's classifier is a Sequential block: [0]=Dropout, [1]=Linear
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

model = model.to(device)

# 5. Loss and Optimizer
criterion = nn.CrossEntropyLoss()
# Optimize ONLY the classifier parameters (since backbone is frozen)
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model if it's the best one so far
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, val_acc_history

In [ ]:
print("Starting training...")
model, train_acc, val_acc = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

save_path = '/content/drive/MyDrive/Colab Notebooks/Plant Disease Dataset/models/plant_disease_efficientnetv2.pth'
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(model.state_dict(), save_path)

print(f"Model trained and saved to: {save_path}")

Starting training...
Epoch 1/10
----------
train Loss: 0.8021 Acc: 0.8164
val Loss: 0.3456 Acc: 0.9167

Epoch 2/10
----------
train Loss: 0.3728 Acc: 0.8963
val Loss: 0.2486 Acc: 0.9340

Epoch 3/10
----------
train Loss: 0.3056 Acc: 0.9088
val Loss: 0.1986 Acc: 0.9447

Epoch 4/10
----------
train Loss: 0.2738 Acc: 0.9164
val Loss: 0.1790 Acc: 0.9489

Epoch 5/10
----------
train Loss: 0.2579 Acc: 0.9186
val Loss: 0.1641 Acc: 0.9521

Epoch 6/10
----------
train Loss: 0.2454 Acc: 0.9209
val Loss: 0.1530 Acc: 0.9534

Epoch 7/10
----------
train Loss: 0.2387 Acc: 0.9226
val Loss: 0.1387 Acc: 0.9578

Epoch 8/10
----------
train Loss: 0.2323 Acc: 0.9270
val Loss: 0.1409 Acc: 0.9586

Epoch 9/10
----------
train Loss: 0.2248 Acc: 0.9260
val Loss: 0.1371 Acc: 0.9571

Epoch 10/10
----------
train Loss: 0.2226 Acc: 0.9271
val Loss: 0.1271 Acc: 0.9600

Training complete in 19m 40s
Best val Acc: 0.960046
Model trained and saved to: /content/drive/MyDrive/Colab Notebooks/Plant Disease Dataset/models/